In [70]:
import torch
import torch.nn as nn
import torchvision
from matplotlib import pyplot as plt

# These three should be changed
InChannel = 0

# Start and end of output channels
OutChannelStart = 0
OutChannelEnd = 16

# file path filters
filterFilePath = './Weights/features.0.weight.pt'

# dont't touch
N = 16
m = 8
delt = pow(2,-m)
Q = pow(2, N-1) - 1
intWeights = torch.load(filterFilePath)
floatWeights = intWeights*delt
conv = nn.Conv2d(1, OutChannelEnd - OutChannelStart, kernel_size=11, stride=4, padding=0)
state_dict = conv.state_dict()
a = floatWeights[OutChannelStart:OutChannelEnd,InChannel,:,:]
state_dict['weight'] = floatWeights[OutChannelStart:OutChannelEnd,InChannel,:,:].view(state_dict['weight'].size())
b = state_dict['weight'][OutChannelStart:OutChannelEnd,InChannel,:,:]
state_dict['bias'] = torch.zeros(state_dict['bias'].size())
conv.load_state_dict(state_dict)

In [85]:
ImageSize = 224
LargeSize = 227
img = torchvision.transforms.ToTensor()(plt.imread('timg.jpg'))
img = nn.functional.interpolate(img.view(1,3,650,1200),size=(ImageSize,ImageSize),mode='bilinear')
img = (img/delt).to(torch.int16)
img = img.to(torch.float) * delt
ttt = torch.zeros(1,3,LargeSize,LargeSize)
ttt[:,:,2:226,2:226] = img
img = ttt
output = ((conv(img[:,InChannel,:,:].view(1,1,LargeSize,LargeSize))/delt).to(torch.int16).to(torch.float) * delt).view(-1,55,55)

In [76]:
F = open(str(InChannel) + '.' + str(OutChannelStart) + '.' + str(OutChannelEnd) + '.outputFloat.csv','w+')
strOutputFloat = ''
for i in range(len(output[0])):
    for j in range(len(output[0])):
        for k in range(OutChannelEnd - OutChannelStart):
            strOutputFloat += '%.8f, '%(output[k][i][j])
    strOutputFloat += '\n'
F.write(strOutputFloat)
F.close()

In [79]:
F = open(str(InChannel) + '.' + str(OutChannelStart) + '.' + str(OutChannelEnd) + '.outputInt.csv','w+')
strOutputInt = ''
for i in range(len(output[0])):
    for j in range(len(output[0])):
        for k in range(OutChannelEnd - OutChannelStart):
            strOutputInt += '%6d, '%(output[k][i][j]/delt)
    strOutputInt += '\n'
F.write(strOutputInt)
F.close()

In [83]:
intImg = img/delt
print intImg.size()
F = open('intImg.csv','w+')
intImgFile = ''
for i in range(227):
    for j in range(227):
        for k in range(3):
            intImgFile += '%6d, '%(intImg[0][k][i][j])
    intImgFile += '\n'
F.write(intImgFile)
F.close()

torch.Size([1, 3, 227, 227])


# Important Findings

Padding 2 in the front and padding 1 in the back, making an 224 x 224 image 227 x 227
is the same as padding 2 in both side to make it 228 x 228 when filter is 11 x 11

In [56]:
print (output1 != output3).sum()

tensor(0)


In [47]:
print output1

tensor([[-2.1289, -0.4492, -0.7891,  ..., -0.8945, -0.4180, -1.0195],
        [-1.3945, -0.3750,  0.5742,  ..., -0.6719,  0.2188, -0.5273],
        [-0.6562, -0.4922, -0.5312,  ..., -0.6953,  0.3320, -0.5938],
        ...,
        [-0.5586,  1.1250, -0.3438,  ..., -1.2500,  0.7031,  0.5781],
        [-0.6094,  0.2148, -0.4102,  ..., -0.9102,  0.3633,  0.9414],
        [-0.6523, -0.1016, -0.6016,  ...,  0.1797,  0.8359,  1.3281]])


In [51]:
print output2

tensor([[-2.1289, -0.4492, -0.7891,  ..., -0.8945, -0.4180, -1.0195],
        [-1.3945, -0.3750,  0.5742,  ..., -0.6719,  0.2188, -0.5273],
        [-0.6562, -0.4922, -0.5312,  ..., -0.6953,  0.3320, -0.5938],
        ...,
        [-0.5586,  1.1250, -0.3438,  ..., -1.2500,  0.7031,  0.5781],
        [-0.6094,  0.2148, -0.4102,  ..., -0.9102,  0.3633,  0.9414],
        [-0.6523, -0.1016, -0.6016,  ...,  0.1797,  0.8359,  1.3281]])


In [55]:
print output3

tensor([[-2.1289, -0.4492, -0.7891,  ..., -0.8945, -0.4180, -1.0195],
        [-1.3945, -0.3750,  0.5742,  ..., -0.6719,  0.2188, -0.5273],
        [-0.6562, -0.4922, -0.5312,  ..., -0.6953,  0.3320, -0.5938],
        ...,
        [-0.5586,  1.1250, -0.3438,  ..., -1.2500,  0.7031,  0.5781],
        [-0.6094,  0.2148, -0.4102,  ..., -0.9102,  0.3633,  0.9414],
        [-0.6523, -0.1016, -0.6016,  ...,  0.1797,  0.8359,  1.3281]])


In [59]:
print output4

tensor([[-1.2695, -0.2695, -0.1719,  ..., -0.8164, -0.2578, -0.6328],
        [-0.8750, -0.3438,  0.7422,  ..., -0.5664,  0.2070, -0.3633],
        [-0.2852, -0.5195, -0.6719,  ..., -0.3672,  0.2891, -0.4453],
        ...,
        [ 0.1523,  0.6641, -0.6328,  ..., -0.8320,  0.9414,  0.3750],
        [-0.2344,  0.0391, -0.6953,  ..., -0.6484,  0.5625,  0.9375],
        [ 0.2305,  0.4141,  0.1250,  ...,  0.8867,  1.5000,  1.4297]])
